In [1]:
import os
import sys
import time

os.chdir('../' )
cwd = os.getcwd()
sys.path.append(cwd)

from oligo_designer_toolsuite.IO._database import CustomDB
from oligo_designer_toolsuite.oligo_selection._resolve_overlapping_oligos import ProbesetGenerator
from oligo_designer_toolsuite.oligo_selection._probe_scoring import PadlockProbeScoring, PadlockSetScoring
from oligo_designer_toolsuite.oligo_selection._heuristic_methods import padlock_heuristic_selection

os.chdir('../' )
cwd = os.getcwd()
dir_output  = os.path.join(cwd, 'oligo-designer-toolsuite/output')
print(dir_output)



/home/francesco/Desktop/Work/oligo-designer-toolsuite/output


Initilitze the classes

In [2]:
dir_annotation = '/home/francesco/Desktop/Work/NCBI'
annotation = dir_annotation+'/GCF_000001405.40_GRCh38.p14_genomic.gtf'
sequence = dir_annotation+'/GCF_000001405.40_GRCh38.p14_genomic.fna'
file_oligos = '/home/francesco/Desktop/Work/oligo-designer-toolsuite/output/probes/probes_info.tsv'




In [3]:
database = CustomDB(probe_length_min = 30, probe_length_max = 40, filters = None, dir_output=dir_output, file_annotation=annotation, file_sequence=sequence)
database.read_oligos_DB(format = 'tsv', file_oligos_DB_tsv = file_oligos)
#enalrge teh DB
genes = list(database.oligos_DB.keys())
for gene in genes:
    for i in range(1):
        database.oligos_DB[gene+f'_{i}'] = database.oligos_DB[gene]
padlock_scoring = PadlockProbeScoring(Tm_min = 52, Tm_opt = 60, Tm_max = 67, GC_min = 40, GC_opt = 50, GC_max = 60)
set_scoring = PadlockSetScoring()
n_jobs = 12
probeset_generator = ProbesetGenerator(n_probes_per_gene = 5, 
                                        min_n_probes_per_gene = 2, DB = database, 
                                        probes_scoring = padlock_scoring, 
                                        heurustic_selection = padlock_heuristic_selection, 
                                        set_scoring=set_scoring,
                                        n_jobs=n_jobs
                                    )
start = time.time()
database = probeset_generator.get_probe_sets(n_sets=100)
print(f'time to process: {time.time() - start}')

/home/francesco/Desktop/Work/oligo-designer-toolsuite/oligo_designer_toolsuite/IO/_database.py:59: UserWarning: Species not specified.
  warnings.warn("Species not specified.")
/home/francesco/Desktop/Work/oligo-designer-toolsuite/oligo_designer_toolsuite/IO/_database.py:63: UserWarning: Genome assembly not specified.
  warnings.warn(f"Genome assembly not specified.")
/home/francesco/Desktop/Work/oligo-designer-toolsuite/oligo_designer_toolsuite/IO/_database.py:67: UserWarning: Annotation release not specified.
  warnings.warn(f"Annotation release not specified.")
/home/francesco/Desktop/Work/oligo-designer-toolsuite/oligo_designer_toolsuite/IO/_database.py:71: UserWarning: Annotation source not specified.
  warnings.warn(f"Annotation source not specified.")


<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
time to process: 2.2627978324890137


In [6]:
help(PadlockProbeScoring.apply)

Help on function apply in module oligo_designer_toolsuite.oligo_selection._probe_scoring:

apply(self, probes, probes_indices)
    Scores all the probes using the defiend scoring function. The scores are both saved in the dictionary
    and in a pandas.Series. The latter is generated because it is the fastest way to generate the sets.
    
    :param probes: dictionary containing the probes
    :type probes: dict
    :param probes_indices: list of the indices of the probes used as a reference to keep the ordering of the probes consistent
    :type probes_indices: np.array
    :return: updated dictionary of the probes, series with the computed scores
    :rtype: dict, pandas.Series

